# Listing Tools and input parameters using nanoHUB web API

Install the nanohubRemote Api, it only works with Python 2.7, a patch to support Python 3.X is on its way

In [1]:
#Install nanohubRemote API
#!git clone https://github.com/bhaley/nanoHUB_remote

## Libraries not installed by default on SciServer Containers
The following libraries are not installed by default, they have to be installed before running the notebook the first time. Libraries can be installed by uncommenting the bellow lines



In [2]:
#!pip install hublib
#!pip install qgrid

## Libraries required

* nanoHUB_remote contains basic functions to stablish communication with nanohub webApi
* hublib includes all graphical components that allow user to change paramenters from a GUI directly in the Jupyter notebook
* Ipython are the core libraries 
* ipywidgets is the standart library provided by Jupyter to create GUI

In [3]:
#access rappture tools & interactive matlab plots
import nanoHUB_remote
import hublib.rappture as rappture
import hublib.tool as tool
import hublib.ui as ui
from hublib.ui import Dropdown
from IPython.display import display
from IPython.display import clear_output
from IPython.core.display import display, HTML
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Figure
import plotly.figure_factory as FF
import plotly.graph_objs as go
import ipywidgets as widgets
import sys
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib
from base64 import b64decode, b64encode
import zlib
from skimage import measure
init_notebook_mode(connected=True)

<IPython.core.display.Javascript object>

## Getting the Nanohub Token


In [5]:
run_results = {}
from mysecrets import auth_data        
print ("Loading Nanohub Token..")
run_results['headers'] = nanoHUB_remote.authenticate(auth_data)       


Loading Nanohub Token..


## Getting the Tools list

Following lines request a list of tools installed on nanohub, loaded into a pandas Dataframe and visualized as a HTML table.

In [121]:
import qgrid
import pandas as pd
run_json = nanoHUB_remote.api.do_post(r'https://nanohub.org/api', 'tools/list', {}, run_results['headers'])
data = pd.DataFrame.from_dict(run_json['tools'])
display(HTML(data.to_html()))

,alias,description,supported,title,versions
0,1dchainmd,"1-D Chain of atoms, bases and layers to produc...",0,1-D Chain Dispersions,[15]
1,1dphononbte,Simulate heat transport by solving one dimensi...,0,1-D Phonon BTE Solver,[9]
2,ddsolarcell,Simulate a 1D solar cell of crystalline materi...,0,1D Drift Diffusion Model for Crystalline Solar...,[16]
3,1dfdmht,Simple 1D Finite Difference Method Tool for Un...,0,1D Finite Different Method Conduction Heat Tra...,[28]
4,1dhetero,Poisson-Schrödinger Solver for 1D Heterostruct...,0,1D Heterostructure Tool,[742]
5,fin,Simulate 1-D moving fin model for CVD roll-to-...,0,1D Moving Fin Model,[2]
6,diffusiongame2d,Play a 2-D diffusion game,0,2-D Diffusion Game,[9]
7,avalanche,This tool performs 2D cluster analysis on one ...,0,2D cluster analysis,[13]
8,smdc,Simulation of a 2d molecular gas with specifie...,0,2d Ideal Gas Molecular Dynamics,[16]
9,BeamProp,Calculation of beam propagation in dielectric ...,0,90 Degrees Beam Propagation,[7]


## Getting a tool input parameters

Following lines request a list of options given a specific tool alias. 

SHOWOPTIONS function prints a dropdown list with  tool's aliases, when an user changes the selected alias a new set of parameters is loaded by requesting the info from nanohub

In [122]:
import requests
key_component = {}
for el in run_json['tools']:
    key_component[el['alias']] = el
key = list(key_component.keys())
def find_rec(node, element, result):
     for item in node.findall(element):
         result.append(item)
         find_rec(item, element, result)
     return result

def SHOWOPTIONS( type_sel ):
    global key_component
    clear_output()
    display(v)
    print (key_component[type_sel['new']])
    #display(HTML()
    run_xml = requests.post(r'https://nanohub.org/api/tools/'+type_sel['new']+'/rappturexml' , {}, run_results['headers'])
    run_results['xml_results'] = run_xml.text
    #print (run_results['xml_results'])
    try:
        run_results['xml_tree'] = ET.fromstring(run_results['xml_results'])
        run_results['input'] = run_results['xml_tree'].find('input')
        run_results['inputs'] = []
        find_rec(run_results['input'], '*[@id]', run_results['inputs'])
        run_results['optionlist'] = []   
        for node in run_results['inputs']:
            reg = {'id':node.attrib['id'], 'type':node.tag }
            if (node.find('about/label') is not None):
                reg['label'] =node.find('about/label').text
            else: 
                reg['label'] = ""
            if (node.find('about/description') is not None):
                reg['description'] =node.find('about/description').text
            else: 
                reg['description'] = ""
            if (node.find('default') is not None):
                reg['default_value'] =node.find('default').text
            else: 
                reg['default_value'] = ""
            if(reg['type']!='group' and reg['type']!='image' and reg['type']!='option'and reg['type']!='phase'):
                run_results['optionlist'].append(reg)
        data = pd.DataFrame.from_dict(run_results['optionlist'] )
        display(HTML(data.to_html(columns=['id','label','type','default_value','description'])))        
    except :
        print ("XML returned is not a valid XML file")
    
    
v = Dropdown ("Tools", "", key, key[0] )
v.dd.observe(SHOWOPTIONS, "value", 'change')
SHOWOPTIONS( {'new':key[0]} )


Box(children=(HTML(value='<p data-toggle="popover" title="">Tools</p>'), Dropdown(index=106, options=('1dchain…

{'alias': 'dftqe', 'title': 'DFT calculations with Quantum ESPRESSO', 'description': 'DFT calculations of molecules and solids', 'versions': ['111'], 'supported': 0}


,id,label,type,default_value,description
0,geometry,Premade atomistic structure,loader,Si.xml,Loads pre made files into Atomic Positions and...
1,ppots,,string,None,
2,cartesian,Atomic Coordinates,choice,Fractional,Determines whether atomic positions are given ...
3,structure,Structure type,choice,Determine unitcell (free),Select a structure to be used for the unit cell
4,title,Title of Run,string,Enter Title Here,The title here will be placed in the input fil...
5,indeck,Atomic Structure,string,\n2\nSilicon diamond structure\nSi 0.0 0.0 0.0...,"The first line is number of atoms, the second ..."
6,lattice_vectors,Cell Vectors (A),string,\n-2.715 2.715 0.000\n-2.715 0.000 2.715\n0.00...,Enter 3 by 3 matrix for the lattice parameters...
7,lat_param_a,"Lattice Parameter ""a"" (A)",number,5.43,Enter the lattice parameter a in units of Angs...
8,lat_param_b_a,"Ratio Lattice Parameters ""b/a""",number,1,Enter the ratio between lattice parameters b a...
9,lat_param_c_a,"Ratio Lattice Parameters ""c/a""",number,1,Enter the ratio between lattice parameters c a...
